In [2]:
import numpy as np
import pandas as pd
import cv2
import math
from sensor_projection_utils import *

In [3]:
import pandas as pd
import json

# Read the JSON data from the file
with open('./Data-palm.json', 'r') as f:
    data = json.load(f)

# Create lists to store the values
time_list = []
pos_x_list = []
pos_y_list = []
pos_z_list = []
rot_x_list = []
rot_y_list = []
rot_z_list = []
rot_order_list = []

# Extract values from the JSON data
for item in data:
    time_list.append(item['time'])
    pos_x_list.append(item['pos']['x'])
    pos_y_list.append(item['pos']['y'])
    pos_z_list.append(item['pos']['z'])
    rot_x_list.append(item['rot']['_x'])
    rot_y_list.append(item['rot']['_y'])
    rot_z_list.append(item['rot']['_z'])
    rot_order_list.append(item['rot']['_order'])

# Create DataFrame
df = pd.DataFrame({
    'time': time_list,
    'pos.x': pos_x_list,
    'pos.y': pos_y_list,
    'pos.z': pos_z_list,
    'rot.x': rot_x_list,
    'rot.y': rot_y_list,
    'rot.z': rot_z_list,
    'order': rot_order_list
})




In [4]:
df = df[df.index % 3 == 0]


In [40]:
def direction_vector_from_quaternion(q):
    # Normalize the quaternion
    hmd_vec = q[4:]
    q = q[0:4]

    magnitude = np.linalg.norm(q)
    normalized_q = q / magnitude
    
    # Calculate the components of the forward vector
    x = 2 * (normalized_q[1] * normalized_q[3] - normalized_q[0] * normalized_q[2]) + hmd_vec[0] 
    y = 2 * (normalized_q[2] * normalized_q[3] + normalized_q[0] * normalized_q[1]) + hmd_vec[1]
    z = 1 - 2 * (normalized_q[1]**2 + normalized_q[2]**2)  + hmd_vec[2]
    
    
    
    return [x,y,z] 

def direction_vector_to_pixel(V, resolution):
    """
    Map a normalized direction vector to pixel coordinates on a 2D equirectangular image.

    Parameters:
    - V: The normalized 3D direction vector in left-handed coordinates.
    - resolution: The resolution of the 2D equirectangular image as a tuple (width, height).

    Returns:
    - (x_pixel, y_pixel): The pixel coordinates on the 2D image.
    """
    # Ensure the vector is normalized
    V /= np.linalg.norm(V)

    # Convert vector to spherical coordinates (theta, phi)
    theta = np.arctan2(V[0], -V[2]) 
    phi = np.arcsin(V[1])

    # Map spherical coordinates to equirectangular pixel coordinates
    x_pixel = (theta + np.pi * 3/2 ) / (2 * np.pi) * resolution[0]
    # To get the mirror value of x, we can subtract the original x value from the width of the image    
    x_pixel = resolution[0] - x_pixel
    
    if x_pixel > resolution[0] :
        print(f'X > width  {x_pixel}')
    if x_pixel < 0 :
        print(f'X < 0 ')
        x_pixel = resolution[0] + x_pixel
        print(f'adjusted X =  {x_pixel}')
    
        
    y_pixel = (phi + np.pi / 2) / np.pi * resolution[1]

    # Clamp the pixel coordinates to the resolution
    x_pixel = np.clip(x_pixel, 0, resolution[0] - 1)
    y_pixel = np.clip(y_pixel, 0, resolution[1] - 1)

    return int(x_pixel), int(y_pixel)

In [41]:
import math

def euler_to_quaternion(roll, pitch, yaw):
    """
    Convert Euler angles (roll, pitch, yaw) to a quaternion vector.
    
    Args:
    - roll: Rotation around the x-axis in radians.
    - pitch: Rotation around the y-axis in radians.
    - yaw: Rotation around the z-axis in radians.
    
    Returns:
    - A tuple representing the quaternion (w, x, y, z).
    """
    
    
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)

    w = cr * cp * cy + sr * sp * sy
    x = sr * cp * cy - cr * sp * sy
    y = cr * sp * cy + sr * cp * sy
    z = cr * cp * sy - sr * sp * cy

    return [w, x, y, z]


In [5]:
df['q'] = df.apply(lambda row : euler_to_quaternion(row['rot.x'],row['rot.y'],row['rot.z']), axis=1)


In [6]:
import pandas as pd
import cv2

def draw_trajectory_user(video_path, df_user):
    # Initialiser une liste pour stocker les données
    data = []
    
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Check if the video is opened successfully
    if not video.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    frame_rate = video.get(cv2.CAP_PROP_FPS)
    resolution = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Set codec
    output_video_path = 'trajectory-walid-sun.mp4'
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, resolution)
    
    # Define the width and height of the rectangular field of view
    rect_width = 550  # for example
    rect_height = 300  # for example
    
    # Process each row in df_user
    for index, row in df_user.iloc[:].iterrows():
        # Convert timestamp to frame number
        playback_time = row['time']
        target_frame_number = int(playback_time * frame_rate)
        print(f'frame number {target_frame_number}')
            
        # Set video to the target frame
        video.set(cv2.CAP_PROP_POS_FRAMES, target_frame_number)
        
        # Read the target frame
        success, frame = video.read()
        if not success:
            print(f"Error: Could not read frame at {target_frame_number}.")
            break
        
        # Convert quaternion and position to pixel coordinates
        q = [row['q'][0], row['q'][1], row['q'][2], row['q'][3]]
        
        direction_vect = direction_vect_from_quaternion_in_cartesian(q)
        direction_vect_spherical = vect_cartesian_to_spherical(direction_vect)
        equirect_point_cartesian = project_spherical_vect_to_equirect_point(direction_vect_spherical)
        
        x_pixel,y_pixel = equirect_point_to_pixels(equirect_point_cartesian,resolution)
        
        # Ajouter les données à la liste
        data.append({'frame_number': target_frame_number, 'x': x_pixel, 'y': y_pixel})
        
        # Draw a circle on the frame
        cv2.circle(frame, (int(x_pixel), int(y_pixel)), 15, (0, 255, 0), -1)
        
        # Calculate top left and bottom right points for the rectangle
        #top_left = (int(x - rect_width / 2), int(y - rect_height / 2))
        #bottom_right = (int(x + rect_width / 2), int(y + rect_height / 2))
        
        # Draw the rectangle
        #cv2.rectangle(frame, top_left, bottom_right, (255, 0, 0), 1)
        
        # Write the modified frame into the output file
        out.write(frame)
    
    # Release resources
    video.release()
    out.release()
    print("Video processing complete. Saved to", output_video_path)
    
    # Convertir la liste en DataFrame et sauvegarder dans un fichier CSV
    #df_output = pd.DataFrame(data)
    #df_output.to_csv('trajectory_palm22.csv', index=False)
    #print("Data saved to trajectory_data.csv")



In [7]:
draw_trajectory_user('./rio.mp4', df)

frame number 0
frame number 1
frame number 2
frame number 3
frame number 4
frame number 5
frame number 6
frame number 7
frame number 8
frame number 9
frame number 10
frame number 11
frame number 12
frame number 13
frame number 14
frame number 15
frame number 16
frame number 17
frame number 18
frame number 19
frame number 20
frame number 21
frame number 22
frame number 23
frame number 24
frame number 25
frame number 26
frame number 27
frame number 28
frame number 29
frame number 30
frame number 31
frame number 32
frame number 33
frame number 34
frame number 35
frame number 36
frame number 37
frame number 38
frame number 39
frame number 40
frame number 41
frame number 42
frame number 43
frame number 44
frame number 45
frame number 46
frame number 47
frame number 48
frame number 49
frame number 50
frame number 51
frame number 52
frame number 53
frame number 54
frame number 55
frame number 56
frame number 57
frame number 58
frame number 59
frame number 60
frame number 61
frame number 62
fr

frame number 490
frame number 491
frame number 492
frame number 493
frame number 494
frame number 495
frame number 496
frame number 497
frame number 498
frame number 499
frame number 500
frame number 501
frame number 502
frame number 503
frame number 504
frame number 505
frame number 506
frame number 507
frame number 508
frame number 509
frame number 510
frame number 511
frame number 512
frame number 513
frame number 514
frame number 515
frame number 516
frame number 517
frame number 518
frame number 519
frame number 520
frame number 521
frame number 522
frame number 523
frame number 524
frame number 525
frame number 526
frame number 527
frame number 528
frame number 529
frame number 530
frame number 531
frame number 532
frame number 533
frame number 534
frame number 535
frame number 536
frame number 537
frame number 538
frame number 539
frame number 540
frame number 541
frame number 542
frame number 543
frame number 544
frame number 545
frame number 546
frame number 547
frame number 5

frame number 976
frame number 977
frame number 978
frame number 979
frame number 980
frame number 981
frame number 982
frame number 983
frame number 984
frame number 985
frame number 986
frame number 987
frame number 988
frame number 989
frame number 990
frame number 991
frame number 992
frame number 993
frame number 994
frame number 995
frame number 996
frame number 997
frame number 998
frame number 999
frame number 1000
frame number 1001
frame number 1002
frame number 1003
frame number 1004
frame number 1005
frame number 1006
frame number 1007
frame number 1008
frame number 1009
frame number 1010
frame number 1011
frame number 1012
frame number 1013
frame number 1014
frame number 1015
frame number 1016
frame number 1017
frame number 1018
frame number 1019
frame number 1020
frame number 1021
frame number 1022
frame number 1023
frame number 1024
frame number 1025
frame number 1026
frame number 1027
frame number 1028
frame number 1029
frame number 1030
frame number 1031
frame number 103

frame number 1441
frame number 1442
frame number 1443
frame number 1444
frame number 1445
frame number 1446
frame number 1447
frame number 1448
frame number 1449
frame number 1451
frame number 1452
frame number 1453
frame number 1454
frame number 1455
frame number 1456
frame number 1457
frame number 1458
frame number 1459
frame number 1460
frame number 1461
frame number 1462
frame number 1463
frame number 1464
frame number 1465
frame number 1467
frame number 1468
frame number 1469
frame number 1470
frame number 1471
frame number 1472
frame number 1473
frame number 1474
frame number 1475
frame number 1476
frame number 1478
frame number 1479
frame number 1480
frame number 1481
frame number 1483
frame number 1484
frame number 1485
frame number 1487
frame number 1488
frame number 1489
frame number 1491
frame number 1492
frame number 1493
frame number 1494
frame number 1495
frame number 1497
frame number 1498
frame number 1499
frame number 1500
frame number 1501
frame number 1502
frame numb

frame number 1979
frame number 1981
frame number 1982
frame number 1983
frame number 1985
frame number 1986
frame number 1987
frame number 1989
frame number 1990
frame number 1991
frame number 1993
frame number 1994
frame number 1995
frame number 1996
frame number 1998
frame number 1999
frame number 2000
frame number 2002
frame number 2003
frame number 2004
frame number 2006
frame number 2007
frame number 2008
frame number 2009
frame number 2011
frame number 2012
frame number 2013
frame number 2015
frame number 2016
frame number 2017
frame number 2018
frame number 2020
frame number 2021
frame number 2022
frame number 2024
frame number 2025
frame number 2026
frame number 2027
frame number 2029
frame number 2030
frame number 2031
frame number 2032
frame number 2034
frame number 2035
frame number 2036
frame number 2038
frame number 2039
frame number 2040
frame number 2041
frame number 2043
frame number 2044
frame number 2045
frame number 2047
frame number 2048
frame number 2049
frame numb

frame number 2602
frame number 2603
frame number 2605
frame number 2606
frame number 2608
frame number 2609
frame number 2611
frame number 2612
frame number 2614
frame number 2615
frame number 2617
frame number 2618
frame number 2620
frame number 2621
frame number 2623
frame number 2624
frame number 2626
frame number 2627
frame number 2629
frame number 2630
frame number 2632
frame number 2633
frame number 2635
frame number 2636
frame number 2638
frame number 2639
frame number 2641
frame number 2642
frame number 2644
frame number 2645
frame number 2647
frame number 2648
frame number 2650
frame number 2651
frame number 2653
frame number 2654
frame number 2656
frame number 2657
frame number 2659
frame number 2660
frame number 2662
frame number 2663
frame number 2665
frame number 2666
frame number 2668
frame number 2669
frame number 2671
frame number 2673
frame number 2674
frame number 2676
frame number 2677
frame number 2679
frame number 2680
frame number 2682
frame number 2683
frame numb

frame number 3483
frame number 3485
frame number 3487
frame number 3489
frame number 3491
frame number 3493
frame number 3495
frame number 3497
frame number 3499
frame number 3501
frame number 3503
frame number 3505
frame number 3507
frame number 3509
frame number 3511
frame number 3513
frame number 3515
frame number 3517
frame number 3519
frame number 3521
frame number 3523
frame number 3525
frame number 3527
frame number 3529
frame number 3531
frame number 3533
frame number 3535
frame number 3537
frame number 3539
frame number 3541
frame number 3543
frame number 3545
frame number 3547
frame number 3549
frame number 3551
frame number 3553
frame number 3555
frame number 3557
frame number 3559
frame number 3561
frame number 3563
frame number 3565
frame number 3567
frame number 3569
frame number 3571
frame number 3573
frame number 3575
frame number 3577
frame number 3579
frame number 3581
frame number 3583
frame number 3585
frame number 3587
frame number 3589
frame number 3591
frame numb

In [22]:
# Open the video file
video = cv2.VideoCapture('./rio.mp4')

# Check if the video is opened successfully
if not video.isOpened():
    print("Error: Could not open video.")

# Get video properties
frame_rate = video.get(cv2.CAP_PROP_FPS)
resolution = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))

In [23]:
resolution

(1920, 960)

In [24]:
import pandas as pd

def normalize_coordinates(csv_file_path, resolution):
    # Charger le fichier CSV
    df = pd.read_csv(csv_file_path)
    
    # Normaliser les colonnes x et y
    df['x'] = df['x'] / resolution[0]
    df['y'] = df['y'] / resolution[1]
    
    # Sauvegarder le fichier CSV normalisé
    normalized_csv_file_path = csv_file_path.replace('.csv', '_normalized.csv')
    df.to_csv(normalized_csv_file_path, index=False)
    
    return normalized_csv_file_path

In [25]:
normalize_coordinates('trajectory_ramzi.csv', resolution)

'trajectory_ramzi_normalized.csv'